# Comparison of temperature and salinity sections along the OVIDE campaign

 - the coordinates of the OVIDE section : coord_OVIDE.txt
 - date of the campaign : 23/06/2012 to 20/07/2012
 - the OVIDE data are in : /scratch/cnt0024/hmg2840/albert7a/OVIDE/
 

In [1]:
!pip install metpy


     |████████████████████████████████| 327kB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 194kB 4.5MB/s eta 0:00:01


In [3]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_sections',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(420)
cluster

from dask.distributed import Client
client = Client(cluster)
client



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.4:42723 Dashboard: http://172.30.100.4:46173/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
!squeue -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9803138     hsw24 make_sec albert7a  R       0:00      1 n2673
           9803139     hsw24 make_sec albert7a  R       0:00      1 n2999
           9803140     hsw24 make_sec albert7a  R       0:00      1 n2566
           9803141     hsw24 make_sec albert7a  R       0:00      1 n2580
           9803142     hsw24 make_sec albert7a  R       0:00      1 n2229
           9803143     hsw24 make_sec albert7a  R       0:00      1 n2238
           9803144     hsw24 make_sec albert7a  R       0:00      1 n2239
           9803145     hsw24 make_sec albert7a  R       0:00      1 n2494
           9803146     hsw24 make_sec albert7a  R       0:00      1 n2495
           9803147     hsw24 make_sec albert7a  R       0:00      1 n2496
           9803148     hsw24 make_sec albert7a  R       0:00      1 n2620
           9803149     hsw24 make_sec albert7a  R       0:00      1 n2621
           9803150     hsw2

In [2]:
import xarray as xr

import glob
import os

import numpy as np
import numpy.matlib
import numpy.ma as ma

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import pyproj
from pyproj import Geod
geod = Geod(ellps='WGS84')

%matplotlib inline

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


## Define the OVIDE domain in eNATL60 data : in notebook sections-OVIDE/2020-03-04-AA-domain-OVIDE-in-eNATL60.ipynb

Results : 
 - imin = 2916
 - imax = 5296
 - jmin = 2257
 - jmax = 4066

In [5]:
## Dataset

gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/coordinates_eNATL60.nc'
maskfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6.nc'


In [6]:
dsgrid=xr.open_dataset(gridfile)
nav_lon=dsgrid.nav_lon
nav_lat=dsgrid.nav_lat

In [7]:
coord_secOV_file='/scratch/cnt0024/hmg2840/albert7a/OVIDE/coord_section_OVIDE2012-2.nc'
dsOV=xr.open_dataset(coord_secOV_file)
latOV=dsOV.latitude
lonOV=dsOV.longitude

In [8]:
jmin = 2257
jmax = 4066
imin = 2916
imax = 5296


In [9]:
nav_lon_OVIDE=nav_lon[jmin:jmax,imin:imax]
nav_lat_OVIDE=nav_lat[jmin:jmax,imin:imax]

# Extract eNATL60 OVIDE domain and period

In [10]:
dir='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02*-S/'
tfiles06='*/eNATL60-BLBT02*_gridT_201006??-201006??.nc'
tfiles07='*/eNATL60-BLBT02*_gridT_201007??-201007??.nc'

list_tfiles06=sorted(glob.glob(dir+tfiles06))
list_tfiles07=sorted(glob.glob(dir+tfiles07))

list_tfiles06_07=list_tfiles06+list_tfiles07

sfiles06='*/eNATL60-BLBT02*_gridS_201006??-201006??.nc'
sfiles07='*/eNATL60-BLBT02*_gridS_201007??-201007??.nc'

list_sfiles06=sorted(glob.glob(dir+sfiles06))
list_sfiles07=sorted(glob.glob(dir+sfiles07))

list_sfiles06_07=list_sfiles06+list_sfiles07


In [11]:
dst=xr.open_mfdataset(list_tfiles06_07,chunks={'x':700,'y':700,'deptht':1})
dstsel=dst.sel(time_counter=slice('2010-06-23','2010-07-20'))
temp_mean_OVIDE=dstsel.votemper[:,:,jmin:jmax,imin:imax].mean(dim='time_counter')


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/backends/api.py:933: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [12]:
lon_debut=lonOV[0]
lat_debut=latOV[0]
lon_fin=lonOV[1]
lat_fin=latOV[1]


In [13]:
start=(lon_debut,lat_debut)
end=(lon_fin,lat_fin)

In [25]:
x=xr.DataArray(np.arange(8354),dims='x',name='x')

In [27]:
dsx=xr.Dataset({'x':x})

In [29]:
y=xr.DataArray(np.arange(4729),dims='y',name='y')

In [40]:
data2=xr.merge([dstsel,dsx,dsy])

In [41]:
data2

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, deptht: 300, time_counter: 672, x: 8354, y: 4729)
Coordinates:
  * x                     (x) int64 0 1 2 3 4 5 ... 8349 8350 8351 8352 8353
  * y                     (y) int64 0 1 2 3 4 5 ... 4724 4725 4726 4727 4728
    nav_lat               (y, x) float32 dask.array<chunksize=(700, 700), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(700, 700), meta=np.ndarray>
  * deptht                (deptht) float32 0.48045477 1.5587916 ... 6025.346
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 2010-06-23T00:30:00 ... 2010-07-20T23:30:00
Dimensions without coordinates: axis_nbounds
Data variables:
    deptht_bounds         (time_counter, deptht, axis_nbounds) float32 dask.array<chunksize=(24, 1, 2), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(24, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(24, 2), meta=np.ndarray>
    votemper              (time_counter, deptht, y, x) float32 dask.array<chunksize=(24, 1, 700, 700), meta=np.ndarray>

In [42]:
from metpy.interpolate import cross_section
data = data2.metpy.parse_cf().squeeze()


Found latitude/longitude values, assuming latitude_longitude for projection grid_mapping variable


In [43]:
cross=cross_section(data, start, end).set_coords(('nav_lat', 'nav_lon'))

ValueError: Data missing required coordinate information. Verify that your data have been parsed by MetPy with proper x and y dimension coordinates and added crs coordinate of the correct projection for each variable.

In [35]:
from metpy.cbook import get_test_data


In [44]:
data = xr.open_dataset(get_test_data('narr_example.nc', False))
data = data.metpy.parse_cf().squeeze()
print(data)


<xarray.Dataset>
Dimensions:              (isobaric: 29, x: 292, y: 118)
Coordinates:
    time                 datetime64[ns] 1987-04-04T18:00:00
  * isobaric             (isobaric) float64 1e+03 975.0 950.0 ... 125.0 100.0
  * y                    (y) float64 -3.087e+06 -3.054e+06 ... 7.114e+05
  * x                    (x) float64 -3.977e+06 -3.945e+06 ... 5.47e+06
    crs                  object Projection: lambert_conformal_conic
Data variables:
    Temperature          (isobaric, y, x) float32 ...
    Lambert_Conformal    |S1 ...
    lat                  (y, x) float64 ...
    lon                  (y, x) float64 ...
    u_wind               (isobaric, y, x) float32 ...
    v_wind               (isobaric, y, x) float32 ...
    Geopotential_height  (isobaric, y, x) float32 ...
    Specific_humidity    (isobaric, y, x) float32 ...
Attributes:
    Conventions:              CF-1.0
    Originating_center:       US National Weather Service - NCEP(WMC) (7)
    Originating_subcenter:    The

In [38]:
start = (37.0, -105.0)
end = (35.5, -65.0)

In [39]:
cross = cross_section(data, start, end).set_coords(('lat', 'lon'))
